# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import re
import pickle
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to C:\Users\wacke-
[nltk_data]     al\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\wacke-
[nltk_data]     al\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\wacke-
[nltk_data]     al\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
X = df['message']
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """ Normalize, tokenize, remove stop words, and lemmatize text string
    Input: text as string, containing the message
    Output: cleaned string list
    """
    # Normalize text: remove URLs
    text = re.sub(r"(https?:\/\/|www\.)\S*", "urlplaceholder", text, re.MULTILINE)    
    
    # Normalize text: remove punctuation characters and capitalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())       
    
    # Tokenize words by splitting the message into a sequence of words
    words = word_tokenize(text)
    
    # Remove stop words, such as "a", "and", "the", "of"
    words = [w for w in words if w not in stopwords.words("english")]    
    
    # Lemmatization: Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    # Lemmatize verbs by specifying pos (e.g., studying --> study)
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Pipeline: Random Forest Classifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# Train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023DF75BCE18>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred_test = pipeline.predict(X_test)

In [9]:
def evaluate_model(y_test, y_pred_test):
    """ Calculates the precision, recall, and f1-score for all output category    
    Input: test data as an array containing actual labels (y_test) and predicted labels (y_pred_test)
    Output: prints the classification report
    """
    for col_no, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred_test[:, col_no]))
        
evaluate_model(y_test, y_pred_test)

related
              precision    recall  f1-score   support

           0       0.70      0.40      0.51      1513
           1       0.84      0.95      0.89      5041

    accuracy                           0.82      6554
   macro avg       0.77      0.68      0.70      6554
weighted avg       0.81      0.82      0.80      6554

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5410
           1       0.81      0.49      0.61      1144

    accuracy                           0.89      6554
   macro avg       0.86      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.00      0.00      0.00        36

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\Users\wacke-al\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wacke-al\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wacke-al\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5798
           1       0.82      0.60      0.69       756

    accuracy                           0.94      6554
   macro avg       0.89      0.79      0.83      6554
weighted avg       0.94      0.94      0.93      6554

shelter
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5959
           1       0.83      0.35      0.49       595

    accuracy                           0.93      6554
   macro avg       0.88      0.67      0.73      6554
weighted avg       0.93      0.93      0.92      6554

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6449
           1       0.80      0.08      0.14       105

    accuracy                           0.98      6554
   macro avg       0.89      0.54      0.57      6554
weighted avg       0.98      0.98      0.98      6554

mo

C:\Users\wacke-al\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wacke-al\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wacke-al\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000023DF75BCE18>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000023DF75BCE18>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [11]:
parameters =  {
    'clf__estimator__n_estimators': [50, 100], # number of trees
    'clf__estimator__min_samples_split': [5, 10, 20] # minimum number of samples required to split a node
} 
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
improved_model = cv.fit(X_train, y_train)

In [ ]:
improved_model.best_params_

In [ ]:
y_pred_test_cv = cv.predict(X_test)
evaluate_model(y_test, y_pred_test_cv)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Try using AdaBoostClassifier instead of Random Forest Classifier
pipeline_2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [ ]:
parameters_2 = {
    'clf__estimator__learning_rate': [0.1, 0.5, 1],
    'clf__estimator__n_estimators': [50, 100, 200]
}
cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2)

In [ ]:
pipeline_2.get_params()

In [ ]:
cv_2.fit(X_train, y_train)

In [ ]:
cv_2.best_params_

In [ ]:
y_pred_test_cv_2 = cv_2.predict(X_test)
evaluate_model(y_test, y_pred_test_cv_2)

As this model does not show better results, stick to the improved model (using RandomForestClassifier)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.